[Хороший шаблон](https://twitter.com/dsunderhood/status/1371874842664910849), который можно использовать, 
когда ваш ноутбук предстоит показать кому-то ещё 
(ну или посмотреть самим через 2 недели).



# Кратко об этом исследовании
Данные из Остинского центра животных, то есть приюта, -- с 1 октября 2013 по **март 2016**. 

### Цель

Требуется предсказать судьбу каждого животного по данным о нём сведениям. По сути, обычная задача категоризации. Классы: Adoption, Died, Euthanasia, Return to owner, Transfer. 

Все классы считаем одинаково важными все зависимости от представленности в выборке. Качество предсказаний оценивается поэтому с помощью macro-averaged F1 score.

---

**Задание**

Пользоваться в точности предложенной в этом шаблоне схемой необязательно, но в этом ноутбуке должен быть 
- внятный и чистый, 
- прокомментированный, 
- воспроизводимый (зафиксируйте все random seeds, которые возможно),
- мотивированный
**код**, который **генерирует ваше лучшее решение**.

А также пока **неформальный отчёт** о проделанной работе.


### Методы

`TODO: Напишите, как пробовали предобрабатывать признаки`

`TODO:Напишите, какие модели и с какими параметрами вы пробовали`


### Результаты

`TODO: Поделитесь наблюдениями, историями успеха и зря потраченными усилиями; что интересного можете сказать о наборе данных? какие выводы?`

---
---
---

## Конфиги и константы
(пожалуйста, без волшебных чисел в коде)


In [1]:
OUTCOME2LABEL = {"Adoption" : 0, 
                 "Transfer": 1, 
                 "Return_to_owner": 2, 
                 "Euthanasia": 3, 
                 "Died": 4
                }
LABEL2OUTCOME = {v: k for k,v in OUTCOME2LABEL.items()}
FOLD_K = 4

## Библиотеки
(все импорты желательно должны быть здесь)

In [4]:
! pip install -q eli5

In [2]:
import sklearn
sklearn.__version__

'0.23.1'

In [5]:
import pandas as pd
import numpy as np
import eli5

from scipy import sparse
from sklearn import tree
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import GridSearchCV

In [7]:
df_train = pd.read_csv("data/train.csv", encoding="utf-8")
df_test = pd.read_csv("data/test.csv", encoding="utf-8")
df_train.head(5)

,Name,SexuponOutcome,AnimalType,AgeuponOutcome,Breed,Color,DateTime,Outcome,ID
0,Socks,Neutered Male,Cat,2 months,Domestic Shorthair Mix,Black/White,2014-06-11 14:36:00,0,0
1,Vera,Intact Female,Cat,1 month,Domestic Shorthair Mix,Tortie/White,2014-07-18 08:10:00,3,1
2,Biscuit,Neutered Male,Dog,3 months,Chihuahua Shorthair Mix,Yellow,2016-01-02 17:28:00,2,2
3,Kitten,Spayed Female,Cat,2 years,Domestic Shorthair Mix,Calico,2014-02-19 17:27:00,0,3
4,NaN,Neutered Male,Cat,2 months,Domestic Shorthair Mix,Orange Tabby,2014-07-21 17:34:00,0,4


### Подготовка признаков

#### Даты

In [8]:
def pandas_dates2number(date_series: pd.Series):
    return pd.to_datetime(date_series).values.astype(np.int64) // 10 ** 9

pandas_dates2number(pd.Series(["2020-12-10"]))

array([1607558400])

#### Возраст

???

#### Цвета и породы

Повторяющихся категорий много, так что можно закодировать их и так, однако **некоторые из них можно и растащить на части**

Необязательно, но можете попробовать, вдруг поможет :)

In [9]:
df_train["Color"].value_counts()

Black/White           1955
Black                 1594
Brown Tabby           1122
Brown Tabby/White      680
White                  649
                      ... 
Gold/Gold                1
White/Black Smoke        1
Silver Lynx Point        1
Brown Tiger/White        1
Calico/Brown Tabby       1
Name: Color, Length: 326, dtype: int64

##### Подсказка: CountVectorizer
Прошу любить и жаловать: векторизация текста из коробки. 

Откройте документацию, если не сталкивались: https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html



In [10]:
vectorizer = CountVectorizer()
vectorized_color = vectorizer.fit_transform(df_train["Color"])

# А много ли получилось слов, описывающих цвета?
print(vectorizer.vocabulary_)

vectorized_color

{'black': 2, 'white': 34, 'tortie': 32, 'yellow': 35, 'calico': 7, 'orange': 18, 'tabby': 27, 'tan': 28, 'buff': 6, 'brown': 5, 'brindle': 4, 'chocolate': 8, 'blue': 3, 'tricolor': 33, 'red': 21, 'torbie': 31, 'sable': 23, 'cream': 9, 'merle': 17, 'gray': 13, 'lynx': 16, 'point': 20, 'flame': 11, 'lilac': 14, 'gold': 12, 'seal': 24, 'silver': 25, 'smoke': 26, 'fawn': 10, 'tick': 29, 'apricot': 1, 'liver': 15, 'tiger': 30, 'agouti': 0, 'pink': 19, 'ruddy': 22}


<18710x36 sparse matrix of type '<class 'numpy.int64'>'
	with 33576 stored elements in Compressed Sparse Row format>

То есть можно векторизовать колонку, а потом эти признаки добавить в общей матрице признаков. Обратите внимание, что если мы к трейну применяем `fit_transform`, то к тесту нужно применить **тот же самый объект-векторизатор** и его метод `transform`.

In [ ]:
# more ideas?

#### Пол

Пола, как мы видим, четыре: стерилизованные и нестерилизованные самки и самцы. Также пол может быть неизвестен.

Может, факт стерилизации стоит сделать отдельным признаком? Но это неточно.

In [11]:
df_train["SexuponOutcome"].value_counts()

Neutered Male    6802
Spayed Female    6223
Intact Female    2466
Intact Male      2438
Unknown           780
Name: SexuponOutcome, dtype: int64

##### Подсказка: OrdinalEncoder

Давайте на примере пола животного и чего-нибудь ещё опробуем OrdinalEncoder

In [12]:
ordinal_encoder = OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1)
ordinal_encoder.fit(df_train[["AnimalType", "Color"]])
ordinal_encoder.transform(df_train[["AnimalType", "Color"]])

TypeError: __init__() got an unexpected keyword argument 'handle_unknown'

Первая колонка закодировалась нулями и единицами (потому что видов животных всего два).

Вторая -- идентификаторами многих уникальных значений поля "Color".

### Единая матрица признаков

In [13]:
def prepare_features(df, oh_encoder=None):
    
    columns_categorical = ["AnimalType", "SexuponOutcome"]
    
    if oh_encoder is None:
        oh_encoder = OneHotEncoder(handle_unknown='ignore') # неизвестные значения на тесте будем игнорировать
        oh_encoder.fit(df[columns_categorical])
    
    encoded_categorical_features = oh_encoder.transform(df[columns_categorical])
    cat_feature_names = oh_encoder.get_feature_names(columns_categorical)
    
    # todo: Огромные числа, которые явно требуют масштабирования/нормализации, right? :) 
    # todo: (ну, как минимум если у вас линейные модели с регуляризаторами) 
    # todo: E.g. если будете вычитать среднее, не забывайте, что из тестовых данных 
    # todo: надо вычесть то же число (то есть придётся сохранить на трейне и передать на тест)! 
    dates = pandas_dates2number(df["DateTime"])
    dates = dates[:, np.newaxis] # делаем массив "двумерным" для последующей склейки
    
    X = sparse.hstack([encoded_categorical_features, dates])    
    
    return X, list(cat_feature_names) + ["date"], oh_encoder


In [14]:
X_train, fnames, ohe_hot_encoder = prepare_features(df_train)
y_train = df_train["Outcome"]
X_test, _, _  = prepare_features(df_test, ohe_hot_encoder)

X_train.shape, X_test.shape, np.array(fnames)

ValueError: Input contains NaN

Мы ещё будем об этом говорить, но в первом приближении советы такие: 
- если разреженные нормализованные признаки (например, если у вас один сплошной OneHotEncoding), есть смысл брать линейные модели;
- если признаков немного (не тысячи), и они, к примеру, даже не нормализованы, есть смысл использовать логические классификаторы -- например, деревья и ансамбли на их основе;
- усреднение/голосование/взвешенное голосование результатов моделей с разными пространствами решений может дать хороший прирост в качестве.

Но всё это с оговорками, конечно.

In [ ]:
# Какие параметры ещё важны для перебора для выбранной вами модели?

param_grid = [
    {"min_samples_leaf": [1, 2, 5],
     "min_samples_split": [2, 5, 10],
     "max_depth": [3, 5, 10],
     "criterion": ["gini", "entropy"]}]
param_grid

In [ ]:
scores = ["f1_macro", "f1_micro"]

for score in scores:
    
    print("# Tuning for %s" % score)
    print()

    # поиск по заданной решётке параметров
    clf = GridSearchCV(tree.DecisionTreeClassifier(random_state=100, class_weight="balanced"),
                       param_grid, 
                       scoring=score, 
                       verbose=1, 
                       # if the estimator is a classifier and y is [...] multiclass, StratifiedKFold is used
                       cv=FOLD_K) 

    # запускаем поиск
    clf.fit(X_train, y_train)

    print("Best params on dev set:")
    print(clf.best_params_)
    
    print("Scores on development set:")
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']

    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))
    
    # обучаем на всём с "лучшими" параметрами
    best_estimator = clf.best_estimator_
    best_estimator.fit(X_train, y_train)
    
    # порождаем и сохраняем сабмит
    y_pred = best_estimator.predict(X_test)
    pd.DataFrame({"ID": df_test["ID"], "Outcome": y_pred}).to_csv("submission_" + score + ".csv", index=None)
    

### Бонус: интервью модели

In [ ]:
eli5.explain_weights(best_estimator, target_names=LABEL2OUTCOME, feature_names=fnames)